In [1]:
import os

os.chdir('/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측')

In [2]:
!ls

 CaliberDataset.py  'Final Mask RCNN.ipynb'   Sample_data
 coco_eval.py	    '이미지 전처리.ipynb'    'Sample_data 적재.ipynb'
 coco_utils.py	    '이미지 적재하기.ipynb'   transforms.py
 Data		     MaskRCNN.ipynb	      utils.py
 engine.py	     __pycache__	      vision


In [3]:
%load CaliberDataset.py

In [4]:
import CaliberDataset
from CaliberDataset import CustomDataset
train = CustomDataset(root = '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data', mode = 'train')
test = CustomDataset(root = '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data', mode = 'test')

100%|██████████| 2/2 [00:02<00:00,  1.11s/it]


In [5]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)

num_classes = 4 + 1 # 원래 클래스 + 1
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [6]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# 분류 목적으로 미리 학습된 모델을 로드하고 특징들만을 리턴하도록 합니다
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# Faster RCNN은 백본의 출력 채널 수를 알아야 합니다.
# mobilenetV2의 경우 1280이므로 여기에 추가해야 합니다.
backbone.out_channels = 1280

# RPN(Region Proposal Network)이 5개의 서로 다른 크기와 3개의 다른 측면 비율(Aspect ratio)을 가진
# 5 x 3개의 앵커를 공간 위치마다 생성하도록 합니다.
# 각 특징 맵이 잠재적으로 다른 사이즈와 측면 비율을 가질 수 있기 때문에 Tuple[Tuple[int]] 타입을 가지도록 합니다.

anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

# 관심 영역의 자르기 및 재할당 후 자르기 크기를 수행하는 데 사용할 피쳐 맵을 정의합니다.
# 만약 백본이 텐서를 리턴할때, featmap_names 는 [0] 이 될 것이라고 예상합니다.
# 일반적으로 백본은 OrderedDict[Tensor] 타입을 리턴해야 합니다.
# 그리고 특징맵에서 사용할 featmap_names 값을 정할 수 있습니다.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)

# 조각들을 Faster RCNN 모델로 합칩니다.
model = FasterRCNN(backbone,
                   num_classes=5,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [7]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256

    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [8]:
# %%shell

# # Download TorchVision repo to use some files from
# # references/detection
# git clone https://github.com/pytorch/vision.git
# cd vision
# git checkout v0.8.2

# cp references/detection/utils.py ../
# cp references/detection/transforms.py ../
# cp references/detection/coco_eval.py ../
# cp references/detection/engine.py ../
# cp references/detection/coco_utils.py ../

In [9]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T
# from torchvision import transforms as T

def get_transform(train):
    transforms = []
    # transforms.append(T.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)))
    transforms.append(T.ToTensor())
    # transforms.append(T.Resize(256))

    # transforms.append(T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)))
    if train:
        # (역자주: 학습시 50% 확률로 학습 영상을 좌우 반전 변환합니다)
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [10]:
import torchvision
import torch

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
train = CustomDataset(root = '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data')
data_loader = torch.utils.data.DataLoader(
    train, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn
)
# For Training
images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]

output = model(images,targets)   # Returns losses and detections
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)     

100%|██████████| 2/2 [00:00<00:00, 17.01it/s]
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [11]:
print(model)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [12]:
# use our dataset and defined transformations
dataset = CustomDataset('/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data')
dataset_test = CustomDataset('/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data', mode = 'test')

# # split the dataset in train and test set
# torch.manual_seed(1)
# indices = torch.randperm(len(dataset)).tolist()
# dataset = torch.utils.data.Subset(dataset, indices[:-50])
# dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=2,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=2, shuffle=False, num_workers=2,
    collate_fn=utils.collate_fn)

100%|██████████| 2/2 [00:00<00:00, 116.53it/s]


In [13]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# our dataset has two classes only - background and person
num_classes = 4 + 1

# get the model using our helper function
model = get_model_instance_segmentation(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.0005
                            , weight_decay=0.005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3, gamma=0.1)

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

In [14]:
# 데이터는 제대로 들어가 있다.
print('train의 길이 :', len(data_loader.dataset))
print('test의 길이 :', len(data_loader_test.dataset))

train의 길이 : 4000
test의 길이 : 400


In [15]:
# let's train it for 10 epochs
from torch.optim.lr_scheduler import StepLR
num_epochs = 3

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)# let's train it for 10 epochs

Epoch: [0]  [   0/2000]  eta: 2:01:11  lr: 0.000001  loss: 3.5093 (3.5093)  loss_classifier: 0.9771 (0.9771)  loss_box_reg: 0.0163 (0.0163)  loss_mask: 2.0746 (2.0746)  loss_objectness: 0.4280 (0.4280)  loss_rpn_box_reg: 0.0133 (0.0133)  time: 3.6356  data: 2.8380  max mem: 1755
Epoch: [0]  [  10/2000]  eta: 1:59:34  lr: 0.000006  loss: 3.5981 (3.6854)  loss_classifier: 0.9771 (0.9752)  loss_box_reg: 0.0142 (0.0288)  loss_mask: 2.1740 (2.2605)  loss_objectness: 0.4280 (0.3988)  loss_rpn_box_reg: 0.0204 (0.0221)  time: 3.6053  data: 3.2919  max mem: 1863
Epoch: [0]  [  20/2000]  eta: 1:19:46  lr: 0.000011  loss: 3.3690 (3.4440)  loss_classifier: 0.9371 (0.9437)  loss_box_reg: 0.0199 (0.0326)  loss_mask: 2.0395 (2.0283)  loss_objectness: 0.4602 (0.4183)  loss_rpn_box_reg: 0.0204 (0.0211)  time: 2.3564  data: 2.0950  max mem: 1865
Epoch: [0]  [  30/2000]  eta: 1:07:21  lr: 0.000016  loss: 3.0413 (3.3193)  loss_classifier: 0.8166 (0.8808)  loss_box_reg: 0.0199 (0.0323)  loss_mask: 1.7976 (

AttributeError: ignored

In [ ]:
targets